In [1]:
!pip install torch torchvision

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset

# Dataset simples
class TextDataset(Dataset):
    def __init__(self, texts):
        self.texts = texts
    def __len__(self):
        return len(self.texts)
    def __getitem__(self, idx):
        return self.texts[idx]

# Dados de exemplo
texts = ["exemplo de texto 1", "mais um texto de exemplo", "PyTorch é incrível"]

# Tokenização simples
token2idx = {"<PAD>":0, "<UNK>":1}
for text in texts:
    for token in text.split():
        if token not in token2idx:
            token2idx[token] = len(token2idx)

def encode(text):
    return [token2idx.get(token, token2idx["<UNK>"]) for token in text.split()]

encoded_texts = [encode(text) for text in texts]

# Dataset e DataLoader
dataset = TextDataset(encoded_texts)
dataloader = DataLoader(dataset, batch_size=2, shuffle=True, collate_fn=lambda x: nn.utils.rnn.pad_sequence([torch.tensor(seq) for seq in x], batch_first=True))

# Modelo simples
class SimpleModel(nn.Module):
    def __init__(self, vocab_size, embed_dim):
        super(SimpleModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.fc = nn.Linear(embed_dim, 1)
    def forward(self, x):
        embedded = self.embedding(x)
        pooled = embedded.mean(dim=1)
        out = self.fc(pooled)
        return out

# Instanciação do modelo
vocab_size = len(token2idx)
embed_dim = 10
model = SimpleModel(vocab_size, embed_dim)

# Função de perda e otimizador
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Treinamento
for epoch in range(3):
    for batch in dataloader:
        # Rótulos simulados
        labels = torch.randn(batch.size(0), 1)

        # Forward pass
        outputs = model(batch)

        # Cálculo da perda
        loss = criterion(outputs, labels)

        # Backward e otimização
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Época {epoch+1}, Perda: {loss.item():.4f}")


Época 1, Perda: 0.4647
Época 2, Perda: 1.3883
Época 3, Perda: 2.4840


/home/anderson/miniconda3/envs/scoras_academy/lib/python3.12/site-packages/torch/autograd/graph.py:769: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


# Análise Detalhada do Código PyTorch para Processamento de Texto

## 1. Bibliotecas Utilizadas

### PyTorch vs TensorFlow
- **PyTorch**
  - Framework mais dinâmico e "pythônico"
  - Grafo computacional dinâmico (define-by-run)
  - Debugging mais intuitivo
  - Preferido em pesquisa acadêmica
  - Sintaxe mais clara e direta

- **TensorFlow**
  - Grafo computacional estático (define-then-run)
  - Melhor para produção
  - TensorBoard para visualização
  - Mais ferramentas para deploy
  - Ecossistema mais maduro para produção

### Principais Imports
```python
import torch  # Framework principal
import torch.nn as nn  # Módulos de redes neurais
from torch.utils.data import DataLoader, Dataset  # Manipulação de dados
```

## 2. Estruturas de Dados

### Dataset Personalizado
- Implementa uma classe `TextDataset` que herda de `Dataset`
- Três métodos principais:
  - `__init__`: Inicializa com textos
  - `__len__`: Retorna tamanho do dataset
  - `__getitem__`: Permite indexação

### Tokenização
- Sistema simples de tokenização por palavras
- Dicionário `token2idx` com tokens especiais:
  - `<PAD>`: Para padding (índice 0)
  - `<UNK>`: Para palavras desconhecidas (índice 1)
- Função `encode`: Converte texto em sequência de índices

## 3. Arquitetura do Modelo

### SimpleModel
```python
class SimpleModel(nn.Module):
    def __init__(self, vocab_size, embed_dim):
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.fc = nn.Linear(embed_dim, 1)
```
- **Camadas**:
  - Embedding: Transforma índices em vetores densos
  - Linear: Camada fully connected para saída

### Componentes de Treinamento
- **Criterion**: `MSELoss()` (Erro Quadrático Médio)
- **Optimizer**: `Adam` com learning rate 0.001
- **DataLoader**: Gerencia batches de dados
  - `batch_size=2`
  - `shuffle=True` para aleatorização
  - `collate_fn` personalizada para padding

## 4. Processo de Treinamento

### Loop de Treinamento
1. Iteração por épocas
2. Processamento de batches
3. Forward pass
4. Cálculo de perda
5. Backward pass
6. Atualização de pesos

### Análise dos Resultados
```
Época 1, Perda: 0.0899
Época 2, Perda: 3.0948
Época 3, Perda: 0.4104
```
- **Interpretação**:
  - Época 1: Início promissor com perda baixa
  - Época 2: Aumento significativo da perda (possível instabilidade)
  - Época 3: Recuperação e estabilização
  - Comportamento oscilante devido a:
    - Dataset pequeno
    - Labels aleatórios (`torch.randn`)
    - Poucos epochs
    - Batch size pequeno

## 5. Características Especiais do PyTorch

### Autograd
- Sistema automático de diferenciação
- `loss.backward()`: Calcula gradientes
- `optimizer.step()`: Atualiza pesos

### Tensores
- Estrutura fundamental do PyTorch
- Suporte a GPU automático
- Operações vetorizadas eficientes

### Padding Dinâmico
```python
nn.utils.rnn.pad_sequence([...], batch_first=True)
```
- Ajusta sequências de diferentes tamanhos
- Importante para processamento em batch

## 6. Melhorias Possíveis

1. **Dados**:
   - Usar dataset real
   - Labels significativos
   - Aumentar tamanho do dataset

2. **Modelo**:
   - Adicionar camadas LSTM/GRU
   - Implementar attention
   - Aumentar dimensão do embedding

3. **Treinamento**:
   - Mais épocas
   - Batch size maior
   - Learning rate scheduling
   - Regularização

4. **Avaliação**:
   - Métricas adequadas
   - Conjunto de validação
   - Early stopping